### Import del dataset e print del tipo di ciascuna colonna prima del cleaning

### Nel dataset i -1 corrispondono a dei Missing Values

In [1]:
import pandas as pd
import numpy as np


df = pd.read_csv(f'DM1_game_dataset.csv')
df = df.copy()

#print the types of the columns
print('\n Types of columns \n')
print(df.dtypes)


 Types of columns 

BGGId                    int64
Name                    object
Description             object
YearPublished            int64
GameWeight             float64
ComWeight              float64
MinPlayers               int64
MaxPlayers               int64
ComAgeRec              float64
LanguageEase           float64
BestPlayers              int64
GoodPlayers             object
NumOwned                 int64
NumWant                  int64
NumWish                  int64
NumWeightVotes           int64
MfgPlaytime              int64
ComMinPlaytime           int64
ComMaxPlaytime           int64
MfgAgeRec                int64
NumUserRatings           int64
NumComments              int64
NumAlternates            int64
NumExpansions            int64
NumImplementations       int64
IsReimplementation       int64
Family                  object
Kickstarted              int64
ImagePath               object
Rank:strategygames       int64
Rank:abstracts           int64
Rank:familygames  

### Conversione della colonna "Description" in un set di stringhe (per evitare doppioni)

In [2]:
import re

def clean_Description(text):
    #Gestisce NaN
    if not isinstance(text, str):
        return set()
    #Rimuove simboli come { } [ ] ' , "
    cleaned = re.sub(r"[{}\[\]',\"]", " ", text)
    # Divide in parole e rimuove spazi vuoti
    words = [w for w in cleaned.strip().split() if w]
    return set(words)

df["Description"] = df["Description"].apply(clean_Description)



### Sostituisco gli 0 e i NaN in YearPublished, GameeWeight, ComWeight, ComAgeRec, LangaugeEase, MfgPlaytime e MfgAgeRec con -1

In [3]:
# DEfinisco le colonne su cui fare l'operazione
cols = ["YearPublished", "GameWeight", "ComWeight", 
        "ComAgeRec", "LanguageEase", "MfgPlaytime", "MfgAgeRec"]

# Sostituisci 0 e NaN con -1
df[cols] = df[cols].replace(0, np.nan).fillna(-1)

#Converto le colonne che devono essere intere in int

df[["YearPublished", "MfgPlaytime", "MfgAgeRec"]] = (
    df[["YearPublished", "MfgPlaytime", "MfgAgeRec"]].astype(int)
)



### Verifica che MinPlayers<=MaxPlayers. Nel caso così non fosse scambio i valori delle due colonne

In [4]:
# Maschera per righe dove entrambi > 0 e MinPlayers > ComMaxPlaytime → scambio
mask_swap = (
    (df["MinPlayers"] > 0) &
    (df["MaxPlayers"] > 0) &
    (df["MinPlayers"] > df["MaxPlayers"])
)

# Maschera per righe dove MaxPlayers == 0 → sostituisci con -1
mask_max_fix = (df["MaxPlayers"] == 0)

#  Maschera per righe dove MinPlayers == 0 → sostituisci con -1
mask_min_fix = (df["MinPlayers"] == 0)


# Scambio dei valori
if mask_swap.any():
    print("Righe con MinPlayers > MaxPlayers trovate:")
    print(df.loc[mask_swap, ["MinPlayers", "MaxPlayers"]])
    df.loc[mask_swap, ["MinPlayers", "MaxPlayers"]] = (
        df.loc[mask_swap, ["MaxPlayers", "MinPlayers"]].values
    )

# Correzione MaxPlayers = 0 → -1
if mask_max_fix.any():
    print("Righe con MaxPlayers = 0 (sostituito con -1):")
    print(df.loc[mask_max_fix, ["MinPlayers", "MaxPlayers"]])
    df.loc[mask_max_fix, "MaxPlayers"] = -1

# Correzione MinPlayers = 0 → -1
if mask_min_fix.any():
    print("Righe con MinPlayers = 0 (sostituito con -1):")
    print(df.loc[mask_min_fix, ["MinPlayers", "MaxPlayers"]])
    df.loc[mask_min_fix, "MinPlayers"] = -1


# Messaggio finale
if not (mask_swap.any() or mask_max_fix.any() or mask_min_fix.any()):
    print("Tutti i valori di MinPlayers e MaxPlayers sono coerenti.")


Righe con MaxPlayers = 0 (sostituito con -1):
       MinPlayers  MaxPlayers
206             2           0
209             1           0
268             0           0
604             2           0
640             1           0
...           ...         ...
21082           2           0
21201           2           0
21243           0           0
21246           1           0
21365           2           0

[173 rows x 2 columns]
Righe con MinPlayers = 0 (sostituito con -1):
       MinPlayers  MaxPlayers
268             0          -1
733             0           2
1104            0           2
1333            0          -1
1486            0          -1
1576            0           2
2805            0          -1
3019            0          -1
3522            0          -1
3597            0          -1
3936            0          -1
5372            0          -1
5621            0           1
6189            0          -1
6909            0          -1
8137            0          -1
8866          

### Converte GoodPlayers in set e verifica che BestPlayers sia in GoodPlayers. Ho notato che gli unici valori in GoodPlayers che danno fastidio sono nella forma int + '+', quindi semplicemente rimuovo il '+' e creo la lista. Successivamente verifica se BestPlayers è in GoodPlayers, nel caso non fosse così sostituisce il valore di BestPlayers con un -1.

In [5]:
import ast

# Funzione per convertire in set di interi
def clean_GoodPlayers(s):
    try:
        items = ast.literal_eval(s)  # converte la stringa in lista
    except Exception:
        return set()  # ritorna un set vuoto se fallisce
    
    result = set()
    for item in items:
        try:
            # Rimuove eventuale '+' alla fine e converte in int
            result.add(int(str(item).rstrip('+')))
        except ValueError:
            print(item)  # per debug
            continue  # ignora altri valori non convertibili
    return result

# Applica la conversione
df["GoodPlayers"] = df["GoodPlayers"].apply(clean_GoodPlayers)

# Verifica che BestPlayers sia presente nel set, sennò sostituisce il valore con un -1
df["BestPlayers"] = df.apply(
    lambda row: row["BestPlayers"] if row["BestPlayers"] in row["GoodPlayers"] else -1,
    axis=1
)


### Il codice controlla che i tempi minimi e massimi di gioco (ComMinPlaytime, ComMaxPlaytime) siano coerenti: scambia i valori se sono invertiti, sostituisce gli zeri con -1, e segnala tutte le modifiche effettuate.

In [6]:
# Maschera per righe dove entrambi > 0 e ComMinPlaytime > ComMaxPlaytime → scambio
mask_swap = (
    (df["ComMinPlaytime"] > 0) &
    (df["ComMaxPlaytime"] > 0) &
    (df["ComMinPlaytime"] > df["ComMaxPlaytime"])
)

# Maschera per righe dove ComMaxPlaytime == 0 → sostituisci con -1
mask_max_fix = (df["ComMaxPlaytime"] == 0)

#  Maschera per righe dove ComMinPlaytime == 0 → sostituisci con -1
mask_min_fix = (df["ComMinPlaytime"] == 0)


# Scambio dei valori
if mask_swap.any():
    print("Righe con ComMinPlaytime > ComMaxPlaytime trovate:")
    print(df.loc[mask_swap, ["ComMinPlaytime", "ComMaxPlaytime"]])
    df.loc[mask_swap, ["ComMinPlaytime", "ComMaxPlaytime"]] = (
        df.loc[mask_swap, ["ComMaxPlaytime", "ComMinPlaytime"]].values
    )

# Correzione ComMaxPlaytime = 0 → -1
if mask_max_fix.any():
    print("Righe con ComMaxPlaytime = 0 (sostituito con -1):")
    print(df.loc[mask_max_fix, ["ComMinPlaytime", "ComMaxPlaytime"]])
    df.loc[mask_max_fix, "ComMaxPlaytime"] = -1

# Correzione ComMinPlaytime = 0 → -1
if mask_min_fix.any():
    print("Righe con ComMinPlaytime = 0 (sostituito con -1):")
    print(df.loc[mask_min_fix, ["ComMinPlaytime", "ComMaxPlaytime"]])
    df.loc[mask_min_fix, "ComMinPlaytime"] = -1


# Messaggio finale
if not (mask_swap.any() or mask_max_fix.any() or mask_min_fix.any()):
    print("Tutti i valori di ComMinPlaytime e ComMaxPlaytime sono coerenti.")


Righe con ComMinPlaytime > ComMaxPlaytime trovate:
       ComMinPlaytime  ComMaxPlaytime
3326               20               1
14500              45              30
19239              90              60
20545              25              15
Righe con ComMaxPlaytime = 0 (sostituito con -1):
       ComMinPlaytime  ComMaxPlaytime
6                   0               0
100                45               0
112                 0               0
116                 0               0
135                 0               0
...               ...             ...
21790               0               0
21812               0               0
21817               0               0
21837               0               0
21914               0               0

[780 rows x 2 columns]
Righe con ComMinPlaytime = 0 (sostituito con -1):
       ComMinPlaytime  ComMaxPlaytime
6                   0              -1
112                 0              -1
116                 0              -1
135                 0      

### In NumComments verifica che sono tutti 0 e poi elimina la colonna

In [7]:
# Verifica se tutti i valori sono 0 (ignorando eventuali NaN)
if (df["NumComments"].fillna(0) == 0).all():
    df = df.drop(columns=["NumComments"])
    print("Colonna 'NumComments' rimossa: tutti i valori erano 0.")
else:
    print("Colonna 'NumComments' NON rimossa: contiene valori diversi da 0.")


Colonna 'NumComments' rimossa: tutti i valori erano 0.


# Tutte le modifiche vengono inserite in un DF chiamato cleaned_df.csv

In [8]:
df.to_csv("cleaned_df.csv", index=False)

In [9]:
#print the types of the columns
print('\n Types of columns \n')
print(df.dtypes)


 Types of columns 

BGGId                    int64
Name                    object
Description             object
YearPublished            int64
GameWeight             float64
ComWeight              float64
MinPlayers               int64
MaxPlayers               int64
ComAgeRec              float64
LanguageEase           float64
BestPlayers              int64
GoodPlayers             object
NumOwned                 int64
NumWant                  int64
NumWish                  int64
NumWeightVotes           int64
MfgPlaytime              int64
ComMinPlaytime           int64
ComMaxPlaytime           int64
MfgAgeRec                int64
NumUserRatings           int64
NumAlternates            int64
NumExpansions            int64
NumImplementations       int64
IsReimplementation       int64
Family                  object
Kickstarted              int64
ImagePath               object
Rank:strategygames       int64
Rank:abstracts           int64
Rank:familygames         int64
Rank:thematic     